## Event generation
This notebook illustrates how to automate large sample event generation for a series of accelerator configurations, particle species, etc. The notebook will run three steps:
 - Event generation in _eSTARlight_
 - ASCII file is then converted to ROOT tree
 - Analysis is then run on the output ROOT file and simple histograms are produced

In [1]:
import itertools
import sys, string, os
from math import log10, floor
import matplotlib.pyplot as plt  
import seaborn as sns
home_dir = '/Users/michaellomnitz/Documents/install_test/'
unit_decode = { 'mb.':-3, 'microb.':-6, 'nanob.':-9, 'picob.':-12, 'femtob.':-15}

Some simple funcitons used for utilies and run workflow:
- round_sig: Round a number to a given number of significant figures
- make_input_file: Takes template input file for _eSTARlight_ and sets the desired $Q^2$ range and final state.
- run_prepared_sim: Runs the full pipeline for a prepared input configuration file _slight.in_. This first generates the M.C. sample in _eSTARlight_, then processes the ASCII file to produce ROOT tree. Finally it runs an analysis on the ROOT file.

*Note*: It is assumed that home_dir contains eSTARlight and the compiled executable e_starlight

In [2]:
def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1)

def make_input_file(template_loc, prod_pid, prod_mode):
    print(prod_mode)
    with open(template_loc,'r') as f:
        data=f.readlines()    
    if prod_mode is 'PP':
        print('here')
        data[33] = 'MIN_GAMMA_Q2 = 0. \n'
        data[34] = 'MAX_GAMMA_Q2 = 1. \n'
    elif prod_mode is 'EP':
        data[33] = 'MIN_GAMMA_Q2 = 1.0 \n'
        data[34] = 'MAX_GAMMA_Q2 = 100.0 \n'
    else:
        print('prod_mode not defined')
        return

    data[29] = 'PROD_PID = '+str(prod_pid)+'\n'
    data[28] = 'N_EVENTS = 100000\n'
    with open(home_dir+'slight.in','w') as f:
        f.writelines(data)
        
def run_prepared_sim(pwd_to_out):
    ascii_to_root = 'root -b -q -l '+home_dir+'eSTARlight/trunk/utils/ConvertStarlightAsciiToTree.C'
    # Run eSTARlight
    os.chdir( home_dir )
    os.system( home_dir+'e_starlight > log' )
    os.system(ascii_to_root)
    # Move files
    os.rename(home_dir+'log',
              pwd_to_out+'.txt')
    os.rename(home_dir+'slight.out',
              pwd_to_out+'.out')
    os.rename(home_dir+'starlight.root', 
             pwd_to_out+'.root')
    # 
    os.chdir(home_dir+'eSTARlight/analysis/')
    os.system('sh e_run.sh '+pwd_to_out+'.root')
    os.rename('photon_energy_bands.eps', 
             pwd_to_out+'_photon_energy_bands.eps')
    os.rename('detector_acceptance.eps', 
             pwd_to_out+'_detector_acceptance.eps.eps')
    # Make root file
    os.chdir( home_dir )

## Generate simluation samples
Set up the specific simulation to produce. We set:
 - particle_prod: Final states to simulate
 - accelerator_facility: Accelerator facilities to simulate
 - prod_m: Production mechanism to study, either photoproduction ('PP') or electroproduction ('EP')
Then run the fll simulation

In [3]:
particle_prod = {'rho':113, 'J_psi':443011}
accelerator_facility = ['eRHIC','JLEIC','LHeC',
                        'eRHIC_eA','JLEIC_eA','LHeC_eA']
prod_m = 'PP'

In [5]:
temp_S = home_dir+'eSTARlight/production/templates/slight_template_'
output_loc = home_dir+'eSTARlight/production/event_generation/'
for key in particle_prod:
    print('Starting with ', key)
    for accel in accelerator_facility:
        out_dir = output_loc+accel
        print(' \t -- ',accel)
        if os.path.isdir(out_dir) is False:
            os.system('mkdir '+out_dir)
        make_input_file(temp_S+accel+'.in',particle_prod[key],prod_m)
        run_prepared_sim(out_dir+'/'+key+'_'+prod_m)

Starting with  J_psi
 	 --  JLEIC
PP
here
